# Time To Bet

## Introduction


C'est la 70ème minute dans le classique PSG-OM et vous en êtes sûr : le PSG est certes mené 1-0, mais au vu de sa possession écrasante et de sa quantité pharaonique d'occasions, le score n'en restera pas là. Plus encore, vous allez jusqu'à mettre votre main à couper qu'il y aura 3-1. 
A la 89ème minute, l'histoire vous a donné raison : il ne reste plus que quelques minutes et le PSG mène 3-1. Vous vous mordez les doigts : _vous auriez dû parier_. 

Voilà un truc un peu lyrique pour accorcher le lecteur et après décrire un peu plus


## Prérequis


## Récupérer les données

Il y a deux moments pour la récupération des données : la phase de collecte de données pour l'entraînement du modèle et la phase de collecte des seules données nécessaires à la visualisation.


#### La collecte des données pour l'entraînement du modèle

A l'aide de sélénium, nous nous connectons sur le site matchendirect.fr qui donne les statistiques d'une grande quantité   de matchs en direct dans le monde. La démarche est la suivante :
- **get_stats()** lance au maximum une fois par minute la fonction **select_champ()** qui boucle sur les championnats.
- **select_champ()** ouvre les url de tous les championnats sélectionnés et récupère la liste de tous les matchs en direct (ils ont un marqueur sépcifique dans le code html : *class='sl '*). Si la liste est vide, on passe au championnat suivant, sinon on lance **select_game()**
- **select_game()** récupère les url de tous les matchs en direct d'un championnat et lance **infos_game()** à chaque fois
- **infos_game()** ouvre l'url d'un match donné et récupère son tableau de statistiques pour l'exporter vers un fichier csv appelé *"Equipe1_Equipe2_Minute"* pour le rendre unique.

Habituellement, cette collecte se fait sur les 5 championnats suivants : Ligue 1, Ligue 2, Liga, Serie A et Bundesliga. La totalité des matchs de ces championnats possèdent des statistiques précises. Toutefois, il est possible de rajouter un autre championnat à étudier en inscrivant son lien en argument de **get_stats()**, ou d'enlever un des championnats principaux de la recherche.  

In [ ]:
## pour boucler sur les 5 championnats principaux
get_stats()

In [ ]:
## pour boucler sur les 5 championnats et un autre championnat
get_stats({'Serie B': 'https://www.matchendirect.fr/italie/serie-b/'})

In [ ]:
## pour boucler sur seulement 4 des 5 championnats
get_stats(champs=['L1', 'L2', 'Liga', 'SerieA'])

#### La collecte des données pour la visualisation


## Retraiter les données

Un seul fichier, *'alldata'*, centralise les informations utiles contenues dans tous les fichiers de type *'Equipe1_Equipe2_Minute'*. La démarche pour constituer ce fichier est la suivante :

- **gather_data()** récupère la liste de tous les fichiers csv présents dans le répertoire et lance **final_scores()**, puis **find_scoreless()**, puis **complete_data()** avant de créer le fichier csv *'alldata'* et de l'acutaliser sur github avec **to_git()**
- **final_scores()** renvoie, à partir de la liste des fichiers csv, la liste de ceux qui donnent les statistiques les plus avancées (dans le temps) pour chaque match.
- **find_scoreless()** récupère cette liste pour filtrer les matchs dont les informations ne seront pas exploitables (par exemple, si on ne possède des informations que jusqu'à la 85ème minute, on considèrera que le score final aura de grandes chances d'avoir évolué et on ne prendra pas ce match en compte).
- à ce stade, les données de tous les matchs dont l'issue est fiable sont compilées dans un même DataFrame
- **complete_data()** rajoute des lignes manquantes. En effet, bien souvent, il y a tellement de matchs à étudier en même temps qu'il est impossible de récupérer toutes leurs statistiques en moins d'une minute. Dans ce cas-là, il peut arriver qu'il nous manque les données d'un match pour une minute. On pourrait alors s'en passer mais on perdrait de l'information précieuse pour notre modèle alors qu'on peut très facilement et fiablement la calculer à partir des statistiques obtenues à la minute précédente et la minute suivante. La fonction récupère donc pour chaque match la liste des minutes pour lesquelles les statistiques font défaut et ajoute au DataFrame la ligne manquant à partir de valeurs recalculées. Pour finir, complete_data ajoute la variable 'Time' au DataFrame (en récupérant la minute des statistiques), cela permettra par la suite au modèle de comprendre qu'il devient de moins en probable de marquer n buts plus le match avance. 
- **to_git()** est décrite à la fin du rapport, elle permet ici si on le souhaite d'actualiser automatiquement la version 'alldata' de github.

L'argument *update* de **gather_data()** permet de choisir si l'on préfère recréer le fichier *'alldata'* à partir de 0 (utile si l'on a modifié manuellement les données de certains matchs déjà présents dans *'alldata'*) ou simplement ajouter les données des derniers matchs étudiés.

(Attention, en raison de la grande quantité de données, cette étape est très chronophage).

In [3]:
## à partir de zéro
gather_data('alldata', update=False, to_git_or_not_to_git=False)

NameError: name 'gather_data' is not defined

In [4]:
## à partir de zéro
gather_data('alldata', update=True)

NameError: name 'gather_data' is not defined

## Modélisation


L'objectif du projet étant de prédire le score d'un match à partir de ses statistiques, il nous faut établir un modèle de prédiction. Nous avons choisi la régression logistique multiclasses de ***sklearn*** : pour chaque match, les statistiques de chaque équipe (à une minute précise) seront étudiées séparément afin de prévoir le nombre de buts qu'elle aura marqués à la fin du match. Plus précisément:

- à partir d'un DataFrame contenant les statistiques d'un ou plusieurs matchs, **predict()** lance **prepare_data()**, **regs()** puis **likely_scores()** et renvoie une prédiction de score ainsi que sa probabilité estimée.
- **prepare_data()** divise le DataFrame en deux avec **sep_by_team()** (un DataFrame par équipe, ie par ligne) puis divise chacun de ces deux DataFrames avec **sep_by_time()** en autant de DataFrames qu'il y a de minutes différentes deands. A la fin, nous nous retrouvons donc avec une liste de DataFrames contenant chacun les données statistiques d'une minute pour toutes les équipes à domicile ou toutes les équipes à l'extérieur.
- **regs()** crée un modèle de régression logistique multiclasses pour chaque DataFrame. Chaque DataFrame est standardisé avant que l'on fasse en sorte d'adapter le modèle au DataFrame. Dans le même temps, on récupère 'alldata' dont on ne conserve que les lignes adaptées (même minute que le DataFrame, même type d'équipe) avant de le diviser en un train_set pour entraîner le modèle et un test_set pour tester le modèle. 
- une fois les modèles entraînés sur les données agrégées, on leur demande de prédire les scores de match dont on ne connaît pas l'issue. 
- on récupère les scores potentiels et les probabilités associés à chacun en filtrant les scores impossibles (0-0 score final alors qu'il y a déjà 1-1 à la 70ème par exemple) avec **likely_scores()** et on renvoie le score dont la probabilité est maximale. On peut ici ajouter une fonction qui permet d'ajuster la prédiction du modèle.

En réalité, on ne récupère pas *une* prédiction et *une* probabilité (celles du score final) mais *deux* : la prédiction et la probabilité du nombre de buts inscrits *par chaque équipe*. Nous avons choisi d'utiliser l'hypothèse simplificatrice que la probabilité du score final était égale au produit des probabilités des nombres de buts de chaque équipe (nous n'avons pas pris en compte la dépendance).

In [ ]:
predict(pd.read_csv("Atalanta_Fiorentina_69'"), 'alldata')

## Visualisation

## En plus


Evidemment, nous n'avons pas abouti à ce résultat en *oneshot* et nous avons donc certains morceaux de code qui ne sont plus nécessaires au produit final mais que nous avons jugé intéressants de mentionner ici.

#### to_git()


Plutôt que d'écrire à chaque fois les mêmes commandes pour actualiser un fichier sur github, nous avons créé la fonction **to_git()** qui inscrit ces commandes elle-même. Renseignez-lui les *noms des fichiers* que vous voulez modifier, leur *emplacement*, votre *clef ssh* et son *mot de passe* (stocké dans un fichier en local), et elle fera ce travail fastidieux à votre place. 
Attention : elle ne gère pas les problèmes de conflit et n'est utile que lorsqu'on est le seul à avoir retravaillé un fichier.

#### red_card()

Une erreur s'était glisée dans le code lors de la collecte de la première moitié des données : nous n'avons pas pu collecter d'informations sur les remplacements et les cartons rouges. Rerécupérer les informations sur les remplacements semblait impossible et finalement peu utile (on se doute bien que c'est une information bonus pour prédire le score d'un match), nous avons donc choisi de retirer cette variable de nos données, le temps de se constituer une base de données assez conqéuente pour pouvoir supprimer les données desmatchs pour lesquels nous n'avons pas cette information. En revanche, le nombre de cartons rouges apparaît bien plus déterminant et nous avons donc créé une fonction qui ouvre l'url de chaque match déjà étudié et renvoie ceux dans lesquels un carton rouge a été distribué. Il n'y en avait qu'un. Et il avait été donné en toute fin de match. Nous avons donc rajouté manuellement cette information sur les quelques fichiers (énumérables sur les doigts d'une main) nécessaires. A présent, toutes les statistiques sont bien collectées et cette fonction ne semble plus avoir d'usage immédiat.

In [ ]:
red_card()

## Conclusion